In [0]:
# -*- coding: utf-8 -*-代码有很多tf1版本的，这里使得代码兼容旧版本的，不要直接使用import tensorflow as tf
#旧版本的主要有placeholders、sessions、collections等等。
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import re
import time

Instructions for updating:
non-resource variables are not supported in the long term


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import os
os.chdir('/content/gdrive/My Drive/NLP/Chinese_Word_Segmentation')

In [0]:
# 读取字典,删除 rstrip的作用是去掉string 字符串末尾的指定字符（默认为空格）.
vocab = open('data/msr/msr_training_words.utf8').read().rstrip('\n').split('\n')
vocab = list(''.join(vocab))
stat = {}
for v in vocab:
    stat[v] = stat.get(v, 0) + 1


In [0]:
stat['白']

267

In [0]:
#根据字典的值排序， items() 方法以列表返回可遍历的(键, 值) 元组数组，
#0表示键，1表示值，默认从小到大，reverse后变为从大到小
stat = sorted(stat.items(), key=lambda x:x[1], reverse=True)
vocab = [s[0] for s in stat]#stat是n x 2 的数组，0表示字
# 5167 个字
print(len(vocab))
# 映射
char2id = {c : i + 1 for i, c in enumerate(vocab)}
id2char = {i + 1 : c for i, c in enumerate(vocab)}
tags = {'s': [1, 0, 0, 0], 'b': [0, 1, 0, 0], 'm': [0, 0, 1, 0], 'e': [0, 0, 0, 1]}#表示句子里字的状态有四种
# B是开始begin位置, E是end, 是结束位置, M是middle, 是中间位置, S是singgle, 单独成词的位置, 没有前, 也没有后

5167


In [0]:
len(char2id)

5167

In [0]:
char2id#字到数字的映射
id2char#数字到字的映射

In [0]:
#@title
import numpy as np 
 
x = np.array([3,  1,  2])  
print ('我们的数组是：')
print (x)
print ('\n')
print ('对 x 调用 argsort() 函数：')
y = np.argsort(x)  
print (y)
print ('\n')
print ('以排序后的顺序重构原数组：')
print (x[y])#x[1],x[2],[0] -> 1,2,3
print ('\n')
print ('使用循环重构原数组：')
for i in y:  
    print (x[i], end=" ")

我们的数组是：
[3 1 2]


对 x 调用 argsort() 函数：
[1 2 0]


以排序后的顺序重构原数组：
[1 2 3]


使用循环重构原数组：
1 2 3 

In [0]:
batch_size = 64

def load_data(path):
    data = open(path).read().rstrip('\n')
    # 按标点符号和换行符分隔
    data = re.split('[，。！？、\n]', data)
    
    # 准备数据
    X_data = []
    Y_data = []
    
    for sentence in data:
        sentence = sentence.split(' ')
        X = []
        Y = []
        
        try:
            for s in sentence:
              #strip() 方法用于移除字符串头尾指定的字符（默认为空格或换行符）或字符序列。
                s = s.strip()
                # 跳过空字符
                if len(s) == 0:
                    continue
                # s
                elif len(s) == 1:
                    X.append(char2id[s])#添加了映射的数字
                    Y.append(tags['s'])#表示这个字单独成词
                elif len(s) > 1:
                    # b
                    X.append(char2id[s[0]])
                    Y.append(tags['b'])#表示这个句子的开始
                    # m
                    for i in range(1, len(s) - 1):
                        X.append(char2id[s[i]])
                        Y.append(tags['m'])#表示这个句子的中间状态
                    # e
                    X.append(char2id[s[-1]])
                    Y.append(tags['e'])#表示这个句子的结尾
        except:
            continue
        else:
            if len(X) > 0:
              #一次添加一个句子进去，添加完后X，Y清空
                X_data.append(X)#添加了映射完成的数字
                Y_data.append(Y)#添加了这个句子里各个词的状态
    #numpy.argsort() 函数返回的是数组值从小到大的索引值。
    order = np.argsort([len(X) for X in X_data])#这里是按照句子的长度，返回句子从小到大排列的索引值
    X_data = [X_data[i] for i in order]#从短到长排列
    Y_data = [Y_data[i] for i in order]#状态按照上面的顺序排列
    
    current_length = len(X_data[0])#最短句子的长度
    X_batch = []
    Y_batch = []
    for i in range(len(X_data)):
      #这里若是同等长度的句子放在一个batch里，bacth的大小小于64但是下一条句子的长度不相同则进入下一个batch，如果batch大于64也进行切分
      #核心思想就是让长度相同的句子放在一个batch里
        if len(X_data[i]) != current_length or len(X_batch) == batch_size:
            yield np.array(X_batch), np.array(Y_batch)
            #迭代器的使用减小了内存开销
            current_length = len(X_data[i])
            X_batch = []
            Y_batch = []
            
        X_batch.append(X_data[i])
        Y_batch.append(Y_data[i])

In [0]:
#@title
def fab(max): 
    n, a, b = 0, 0, 1 
    while n < max: 
        yield b      # 使用 yield
        # print b 
        a, b = b, a + b 
        n = n + 1
 
for n in fab(5): 
    print n
#减小了内存占用的开销

In [0]:
#全卷积神经网络fcn，没有池化层，这里可以参考https://zhuanlan.zhihu.com/p/37803719
embedding_size = 128
#产生了（len(char2id) + 1） x embedding_size的矩阵，并限定范围-1到1
embeddings = tf.Variable(tf.random.uniform([len(char2id) + 1, embedding_size], -1.0, 1.0))
#tf.placeholder()函数作为一种占位符用于定义过程，可以理解为形参，在执行的时候再赋具体的值。
X_input = tf.placeholder(dtype=tf.int32, shape=[None, None], name='X_input')#X_input为对应整篇文章句子里字以及字的状态
##b = tf.nn.embedding_lookup(c, [1, 3])#查找数组中的序号为1和3的,具体的可以参照博客https://blog.csdn.net/hit0803107/article/details/98377030详细解读
embedded = tf.nn.embedding_lookup(embeddings, X_input)
#5168x128->到3x128x64
W_conv1 = tf.Variable(tf.random_uniform([3, embedding_size, embedding_size // 2], -1.0, 1.0))
b_conv1 = tf.Variable(tf.random_uniform([embedding_size // 2], -1.0, 1.0))
Y_conv1 = tf.nn.relu(tf.nn.conv1d(embedded, W_conv1, stride=1, padding='SAME') + b_conv1)
#3x128x64->3x64x32
W_conv2 = tf.Variable(tf.random_uniform([3, embedding_size // 2, embedding_size // 4], -1.0, 1.0))
b_conv2 = tf.Variable(tf.random_uniform([embedding_size // 4], -1.0, 1.0))
Y_conv2 = tf.nn.relu(tf.nn.conv1d(Y_conv1, W_conv2, stride=1, padding='SAME') + b_conv2)
#3x64x32->3x32x4
W_conv3 = tf.Variable(tf.random_uniform([3, embedding_size // 4, 4], -1.0, 1.0))
b_conv3 = tf.Variable(tf.random_uniform([4], -1.0, 1.0))
Y_pred = tf.nn.softmax(tf.nn.conv1d(Y_conv2, W_conv3, stride=1, padding='SAME') + b_conv3, name='Y_pred')#SAME表示填充，尺寸不改变

Y_true = tf.placeholder(dtype=tf.float32, shape=[None, None, 4], name='Y_true')
cross_entropy = tf.reduce_mean(-tf.reduce_sum(Y_true * tf.log(Y_pred + 1e-20), axis=[2]))#交叉熵axis=[2]表示在四种状态下的各个交叉熵，这也是和平时加上池化层后不同的地方
optimizer = tf.train.AdamOptimizer().minimize(cross_entropy)#定义优化器Adam()
#利用tf.argmax()按行求出真实值、预测值最大值的下标，也即是逆独热编码后的标签用tf.equal()求出真实值和预测值相等的数量，
#也就是预测结果正确的数量，tf.argmax()和tf.equal()一般是结合着用。
correct_prediction = tf.equal(tf.argmax(Y_pred, 2), tf.argmax(Y_true, 2))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [0]:
saver = tf.train.Saver()
max_test_acc = -np.inf

epochs = 50
sess = tf.Session()
sess.run(tf.global_variables_initializer())
for e in range(epochs):
    train = load_data('data/msr/msr_training.utf8')#分词好的训练集
    accs = []
    i = 0
    t0 = int(time.time())
    for X_batch, Y_batch in train:
        sess.run(optimizer, feed_dict={X_input: X_batch, Y_true: Y_batch})
        i += 1
        if i % 100 == 0:
            acc = sess.run(accuracy, feed_dict={X_input: X_batch, Y_true: Y_batch})
            accs.append(acc)
    print('Epoch %d time %ds' % (e + 1, int(time.time()) - t0))
    print('- train accuracy: %f' % (np.mean(accs)))

    test = load_data('data/msr/msr_test_gold.utf8')#分词好的测试集
    accs = []
    for X_batch, Y_batch in test:
        acc = sess.run(accuracy, feed_dict={X_input: X_batch, Y_true: Y_batch})
        accs.append(acc)
    mean_test_acc = np.mean(accs)
    print('- test accuracy: %f' % mean_test_acc)

    if mean_test_acc > max_test_acc:
        max_test_acc = mean_test_acc
        print('Saving Model......')
        saver.save(sess, './msr_fcn/msr_fcn')

Epoch 1 time 60s
- train accuracy: 0.661292
- test accuracy: 0.670330
Saving Model......
Epoch 2 time 58s
- train accuracy: 0.811814
- test accuracy: 0.774471
Saving Model......
Epoch 3 time 60s
- train accuracy: 0.875887
- test accuracy: 0.821054
Saving Model......
Epoch 4 time 60s
- train accuracy: 0.903602
- test accuracy: 0.838952
Saving Model......
Epoch 5 time 61s
- train accuracy: 0.921127
- test accuracy: 0.803259
Epoch 6 time 61s
- train accuracy: 0.933134
- test accuracy: 0.769136
Epoch 7 time 59s
- train accuracy: 0.942237
- test accuracy: 0.753947
Epoch 8 time 61s
- train accuracy: 0.948014
- test accuracy: 0.744312
Epoch 9 time 61s
- train accuracy: 0.953760
- test accuracy: 0.738377
Epoch 10 time 60s
- train accuracy: 0.957925
- test accuracy: 0.750232
Epoch 11 time 61s
- train accuracy: 0.961451
- test accuracy: 0.753207
Epoch 12 time 59s
- train accuracy: 0.964341
- test accuracy: 0.754849
Epoch 13 time 61s
- train accuracy: 0.966490
- test accuracy: 0.748641
Epoch 14 t

In [0]:
def viterbi(nodes):
    trans = {'be': 0.5, 'bm': 0.5, 'eb': 0.5, 'es': 0.5, 'me': 0.5, 'mm': 0.5, 'sb': 0.5, 'ss': 0.5}#转移概率
    paths = {'b': nodes[0]['b'], 's': nodes[0]['s']}#第一个字要么是单个字的s，要么是一个句子的开始
    for l in range(1, len(nodes)):
        paths_ = paths.copy()
        paths = {}
        for i in nodes[l].keys():
            nows = {}
            for j in paths_.keys():
                if j[-1] + i in trans.keys():
                    nows[j + i] = paths_[j] + nodes[l][i] + trans[j[-1] + i]
            nows = sorted(nows.items(), key=lambda x: x[1], reverse=True)#根据概率从大到小排列
            paths[nows[0][0]] = nows[0][1]
    
    paths = sorted(paths.items(), key=lambda x: x[1], reverse=True)#根据概率从大到小排列
    return paths[0][0]

In [0]:
def cut_words(data):
    data = re.split('[，。！？、\n]', data)
    sens = []
    Xs = []
    for sentence in data:
        sen = []
        X = []
        sentence = list(sentence)#将句子变成字的列表
        for s in sentence:
            s = s.strip()#方法用于移除字符串头尾指定的字符（默认为空格或换行符）或字符序列。
            if not s == '' and s in char2id:#如果字不为空并且字在字表里
                sen.append(s)
                X.append(char2id[s])#将字的映射数字添加到列表里
        
        if len(X) > 0:
            Xs.append(X)#添加句子的映射数字列表
            sens.append(sen)#句子列表
    
    results = ''
    for i in range(len(Xs)):
        X_d = np.array([Xs[i]])#一个句子,有多少个字句子重复多少次
        Y_d = sess.run(Y_pred, feed_dict={X_input: X_d})#
        nodes = [dict(zip(['s', 'b', 'm', 'e'], d)) for d in Y_d[0]]#每个字的状态概率，例如一个长度为10的句子，得到的就是10x4的概率矩阵
        
        ts = viterbi(nodes)#维特比算法求出最大概率转移矩阵
        for x in range(len(sens[i])):
            if ts[x] in ['s', 'e']:
                results += sens[i][x] + '/'
            else:
                results += sens[i][x]
    
    return results[:-1]

In [0]:
#@title
node = [dict(zip(['s', 'b', 'm', 'e'],[1,2,3,4]))]
node

[{'b': 2, 'e': 4, 'm': 3, 's': 1}]

In [0]:
print(cut_words('中国共产党第十九次全国代表大会，是在全面建成小康社会决胜阶段、中国特色社会主义进入新时代的关键时期召开的一次十分重要的大会。'))
print(cut_words('把这本书推荐给，具有一定编程基础，希望了解数据分析、人工智能等知识领域，进一步提升个人技术能力的社会各界人士。'))
print(cut_words('结婚的和尚未结婚的。'))

中国共产党第十九次/全国代表大会/是/在/全面/建成/小康/社会/决胜/阶段/中国/特色/社会主义/进入/新时代/的/关键/时期/召开/的/一次/十分/重要/的/大会
把/这/本/书/推荐/给/具有/一定编程/基础/希望/了解/数据/分析/人工智能/等/知识/领域/进一步/提升/个人/技术/能力/的/社会各界/人士
结婚/的/和/尚未/结婚/的


In [0]:
len(Xs)

59

In [0]:
X_d

array([[  13,    7,   91,  341,   95,   18,  157,  241,  319,  439,   85,
          87,  123,  850,  252, 2746,   87,  254, 1945,  154,  850,   23,
         139,   20,   44,  277,  704,  850,   19,   18]])

In [0]:
Xs[-1] = np.array(Xs[-1])
Xs[-1]

array([  13,    7,   91,  341,   95,   18,  157,  241,  319,  439,   85,
         87,  123,  850,  252, 2746,   87,  254, 1945,  154,  850,   23,
        139,   20,   44,  277,  704,  850,   19,   18])

In [0]:
Xs[0]

[13, 7, 187, 172, 125, 51, 20, 109, 139, 149, 7, 123, 257, 19, 18]

In [0]:
Y_d[0].shape

(30, 4)

In [0]:
data = '中国共产，是在全、中国特色社会。'
data = re.split('[，。！？、\n]', data)
sens = []
Xs = []
for sentence in data:
  sen = []
  a = []
  print(sentence)
  sentence = list(sentence)#将句子变成字的列表
  for s in sentence:
    s = s.strip()#方法用于移除字符串头尾指定的字符（默认为空格或换行符）或字符序列。
    if not s == '' and s in char2id:#如果字不为空并且字在字表里
      sen.append(s)
      a.append(char2id[s])#将字的映射数字添加到列表里
        
    
    Xs.append(a)#添加句子的映射数字列表
    sens.append(sen)#句子列表
    print(Xs)
    


中国共产
[[13]]
[[13, 7], [13, 7]]
[[13, 7, 187], [13, 7, 187], [13, 7, 187]]
[[13, 7, 187, 172], [13, 7, 187, 172], [13, 7, 187, 172], [13, 7, 187, 172]]
是在全
[[13, 7, 187, 172], [13, 7, 187, 172], [13, 7, 187, 172], [13, 7, 187, 172], [605]]
[[13, 7, 187, 172], [13, 7, 187, 172], [13, 7, 187, 172], [13, 7, 187, 172], [605, 669], [605, 669]]
[[13, 7, 187, 172], [13, 7, 187, 172], [13, 7, 187, 172], [13, 7, 187, 172], [605, 669, 149], [605, 669, 149], [605, 669, 149]]
中国特色社会
[[13, 7, 187, 172], [13, 7, 187, 172], [13, 7, 187, 172], [13, 7, 187, 172], [605, 669, 149], [605, 669, 149], [605, 669, 149], [13]]
[[13, 7, 187, 172], [13, 7, 187, 172], [13, 7, 187, 172], [13, 7, 187, 172], [605, 669, 149], [605, 669, 149], [605, 669, 149], [13, 7], [13, 7]]
[[13, 7, 187, 172], [13, 7, 187, 172], [13, 7, 187, 172], [13, 7, 187, 172], [605, 669, 149], [605, 669, 149], [605, 669, 149], [13, 7, 91], [13, 7, 91], [13, 7, 91]]
[[13, 7, 187, 172], [13, 7, 187, 172], [13, 7, 187, 172], [13, 7, 187, 172], [

In [0]:
Y_d.shape#最后一个句子

(1, 30, 4)

In [0]:
print(sens[0])
print(sens[-1])
len(Xs)

['中', '国', '共', '产', '党', '第', '十', '九', '次', '全', '国', '代', '表', '大', '会']
['中', '国', '特', '色', '社', '会', '主', '义', '进', '入', '新', '时', '代', '的', '关', '键', '时', '期', '召', '开', '的', '一', '次', '十', '分', '重', '要', '的', '大', '会']


59

In [0]:
print(cut_words('中国共产党第十九次全国代表大会，是在全面建成小康社会决胜阶段、中国特色社会主义进入新时代的关键时期召开的一次十分重要的大会。'))
print(cut_words('把这本书推荐给，具有一定编程基础，希望了解数据分析、人工智能等知识领域，进一步提升个人技术能力的社会各界人士。'))
print(cut_words('结婚的和尚未结婚的。'))

In [0]:
data = re.split('[，。！？、\n]', '中国共产党第十九次全国代表大会，是在全面建成小康社会决胜阶段、中国特色社会主义进入新时代的关键时期召开的一次十分重要的大会。')
for sentence in data:
  sentence = list(sentence)
  print(sentence)

['中', '国', '共', '产', '党', '第', '十', '九', '次', '全', '国', '代', '表', '大', '会']
['是', '在', '全', '面', '建', '成', '小', '康', '社', '会', '决', '胜', '阶', '段']
['中', '国', '特', '色', '社', '会', '主', '义', '进', '入', '新', '时', '代', '的', '关', '键', '时', '期', '召', '开', '的', '一', '次', '十', '分', '重', '要', '的', '大', '会']
[]


In [0]:
a = np.array(Xs)
a

In [0]:
Xa = np.array(Xs)
Xa.shape[0]

59